In [1]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.1 MB 7.6 MB/s 
     |████████████████████████████████| 2.0 MB 48.6 MB/s 
     |████████████████████████████████| 212 kB 65.4 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
     |████████████████████████████████| 1.1 MB 45.0 MB/s 
     |████████████████████████████████| 256 kB 12.2 MB/s 
     |████████████████████████████████| 54 kB 1.7 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 271 kB 43.3 MB/s 
     |████████████████████████████████| 144 kB 60.9 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 11.1 MB 60.7 MB/s 
     |████████████████████████████████| 80 kB 9.1 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 62 kB

In [2]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 9.5 MB/s 
     |████████████████████████████████| 596 kB 5.8 MB/s 
     |████████████████████████████████| 6.6 MB 46.8 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 8.0 MB/s 


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

In [6]:
tokenizer = AutoTokenizer.from_pretrained("allenai/macaw-large") #300 million paramaters

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained("allenai/macaw-large") #300 million paramaters

Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

In [8]:
import gradio as gr
import os
import numpy as np
os.system("pip install pdfminer.six rank_bm25 torch transformers")

0

In [9]:
from gradio.mix import Series
from rank_bm25 import BM25Okapi
import string 
import torch
from transformers import pipeline
import pdfminer
from pdfminer.high_level import extract_text


In [10]:
len_doc = 500
overlap = 15 
param_top_k_retriver = 15
param_top_k_ranker = 3


In [11]:
qa_model = pipeline("question-answering", 
                    #model = "deepset/minilm-uncased-squad2")
                    model = "deepset/roberta-base-squad2"
                    #model="vblagoje/bart_lfqa"
                    )

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [12]:
def read_pdf(file):
  """
  Function takes in a pdf file and converts it into a text corpus in 
  the form of a list of docs
  Here len_doc=500, overlap=15
  """
  text = extract_text(file.name)
  # Split text into smaller docs
  docs = []
  
  i = 0
  while i < len(text):
    docs.append(text[i:i+len_doc])
    i = i + len_doc - overlap

  return docs

In [13]:
def read_text(text):
  """
  Function converts a readymade text corpus into a list of docs
  Here again len_doc=500, overlap=15
  """
  docs = []
  
  i = 0
  while i < len(text):
      docs.append(text[i:i+len_doc])
      i = i + len_doc - overlap
  return docs

In [14]:
def bm25_tokenizer(text):
  """
  Tokenizer function which tokenizes our text corpus
  """
  #stop_w = ['a', 'the', 'am', 'is' , 'are', 'who', 'how', 'where', 'when', 'why', 'what']
  stop_w = []
  tokenized_doc = []
  for token in text.lower().split():
      token = token.strip(string.punctuation)

      if len(token) > 0 and token not in stop_w:
          tokenized_doc.append(token)
  return tokenized_doc

In [15]:
def retrieval(query, top_k_retriver, docs, bm25_):
    """
Here top_k_retriever=15
Function takes in a query,our text corpus and an object formed
by passing our tokenized corpus to the BM250kapi
The function computes the similiarity scores between the query and 
the corpus,ranks them and returns the top15 docs
along with their scores
    """
    bm25_scores = bm25_.get_scores(bm25_tokenizer(query))
    top_n = np.argsort(bm25_scores)[::-1][:top_k_retriver]
    bm25_hits = [{'corpus_id': idx, 
                  'score': bm25_scores[idx], 
                  'docs':docs[idx]} for idx in top_n if bm25_scores[idx] > 0]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
    
    return bm25_hits

In [16]:
def qa_ranker(query, docs_, top_k_ranker):
  """
  Here top_k_ranker=3
  Takes in the query and the top 15 docs from the retrieval function
  and passes each doc to the model pipeline.The output of the Pipeline
  is again sorted accordind and the top 3 answers are returned
  """
  ans = []
  for doc in docs_:
      answer = qa_model(question = query, 
                            context = doc)
      answer['doc'] = doc
      ans.append(answer)
  return sorted(ans, key=lambda x: x['score'], reverse=True)[:top_k_ranker]

In [17]:
#Functions for text formatting
def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)
def cstr_bold(s, color='black'):
    return "<text style=color:{}><b>{}</b></text>".format(color, s)
def cstr_break(s, color='black'):
    return "<text style=color:{}><br>{}</text>".format(color, s)


In [18]:
def print_colored(text, start_idx, end_idx, confidence):
  """
  Takes in the top 3 inputs from the qa_ranker function
  and highlights the actual answer in the docs from the
  start to the end index 
  """
  conf_str = '- Confidence: ' +  confidence
  a = cstr(' '.join([text[:start_idx], \
                      cstr_bold(text[start_idx:end_idx], color='blue'), \
                      text[end_idx:], \
                      cstr_break(conf_str, color='grey')]), color='black')
  return a

In [19]:
def final_qa_pipeline(file, query):
  """
  File->Text corpus->Tokenised corpus->Retrieving top15 docs based on similarity with query->
  Passing the 15 docs to pipeline and returning the top 3 answers
  """
  docs = read_pdf(file)
  tokenized_corpus = []
  for doc in docs:
      tokenized_corpus.append(bm25_tokenizer(doc))
    
  bm25 = BM25Okapi(tokenized_corpus)
    
  top_k_retriver, top_k_ranker = param_top_k_retriver, param_top_k_ranker
  lvl1 = retrieval(query, top_k_retriver, docs, bm25)

  if len(lvl1) > 0:
      fnl_rank = qa_ranker(query, [l["docs"] for l in lvl1], top_k_ranker)
      top1 = print_colored(fnl_rank[0]['doc'], fnl_rank[0]['start'], fnl_rank[0]['end'], str(np.round(100*fnl_rank[0]["score"],1))+"%")  
      if len(lvl1)>1:
          top2 = print_colored(fnl_rank[1]['doc'], fnl_rank[1]['start'], fnl_rank[1]['end'], str(np.round(100*fnl_rank[1]["score"],1))+"%")
      else:
          top2 = "None"
      return (top1, top2)
  else:
      return ("No match","No match")

In [20]:
def final_qa_pipeline2(context, query):
    docs = read_text(context)
    tokenized_corpus = []
    for doc in docs:
        tokenized_corpus.append(bm25_tokenizer(doc))
    
    bm25 = BM25Okapi(tokenized_corpus)
    
    top_k_retriver, top_k_ranker = param_top_k_retriver, param_top_k_ranker
    lvl1 = retrieval(query, top_k_retriver, docs, bm25)

    if len(lvl1) > 0:
        fnl_rank = qa_ranker(query, [l["docs"] for l in lvl1], top_k_ranker)
        top1 = print_colored(fnl_rank[0]['doc'], fnl_rank[0]['start'], fnl_rank[0]['end'], str(np.round(100*fnl_rank[0]["score"],1))+"%")  
        if len(lvl1)>1:
            top2 = print_colored(fnl_rank[1]['doc'], fnl_rank[1]['start'], fnl_rank[1]['end'], str(np.round(100*fnl_rank[1]["score"],1))+"%")
        else:
            top2 = "None"
        return (top1, top2)
    else:
        return ("No match","No match")

In [21]:
def final_qa_pipeline3(context,query):
  """
  Pipeline to process the short text input and feed it to the 
  roberta model
  """
  d={'question':query,'context':context}
  res=qa_model(d)
  return (res['answer'],res['score'])

In [22]:
def trivia_answer(text):
  """
  Function to take process the trivia question input 
  in the format necessary for the macaw model
  """
  input_string = "$answer$; $explanation$; $question$=" + text
  input_ids = tokenizer.encode(input_string, return_tensors="pt")
  output = model.generate(input_ids, max_length=200)
  answer=tokenizer.batch_decode(output, skip_special_tokens=True)
  return answer

In [23]:
demo = gr.Blocks()

with demo:
    gr.Markdown("Question Answering System: Enter a PDF File or the relevant Context. You can also try basic trivia questions with the triva option.")
    with gr.Tabs():
        with gr.TabItem("PDF Input"):
            file_input=gr.File(label="Input PDF File")
            text_input=gr.inputs.Textbox(label="Question:")
            text_output=[gr.outputs.HTML(label="Top 1 answer"), gr.outputs.HTML(label="Top 2 answer")]
            pdf_button = gr.Button("Answer")
        
        with gr.TabItem("Text Input"):
            text_input1=gr.inputs.Textbox(label="Context:")
            text_input2=gr.inputs.Textbox(label="Question:")
            text_output2=[gr.outputs.HTML(label="Top 1 answer"), gr.outputs.HTML(label="Top 2 answer")]    
            text_button = gr.Button("Answer")
        
        with gr.TabItem("Text Input(Short)"):
            text_input3=gr.inputs.Textbox(label="Context:")
            text_input4=gr.inputs.Textbox(label="Question:")
            text_output3=[gr.outputs.HTML(label="Answer"), gr.outputs.HTML(label="Confidence")]    
            text_button2 = gr.Button("Answer")

        with gr.TabItem("Trivia"):
            input=gr.inputs.Textbox(label="Question:")
            output=gr.outputs.Textbox(label="Answer")    
            text_button3 = gr.Button("Answer")

    pdf_button.click(final_qa_pipeline, inputs=[file_input,text_input], outputs=text_output)
    text_button.click(final_qa_pipeline2, inputs=[text_input1,text_input2], outputs=text_output2)
    text_button2.click(final_qa_pipeline3, inputs=[text_input3,text_input4], outputs=text_output3)
    text_button3.click(trivia_answer, inputs=input, outputs=output)

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


In [24]:
demo.launch(debug=True,share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://36919.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f706bd5a610>,
 'http://127.0.0.1:7860/',
 'https://36919.gradio.app')